# Exercise 9 - Using the GPU API

**GOAL:** The goal of this exercise is to show how to use GPUs with remote functions and actors.

**NOTE:** These exercises are designed to run on a machine without GPUs.

See the documentation on using Ray with GPUs http://ray.readthedocs.io/en/latest/using-ray-with-gpus.html.

### Concepts for this Exercise - Using Ray with GPUs

We can indicate that a remote function or an actor requires some GPUs using the `num_gpus` keyword.

```python
@ray.remote(num_gpus=1)
def f():
    # The command ray.get_gpu_ids() returns a list of the indices
    # of the GPUs that this task can use (e.g., [0] or [1]).
    ray.get_gpu_ids()

@ray.remote(num_gpus=2)
class Foo(object):
    def __init__(self):
        # The command ray.get_gpu_ids() returns a list of the
        # indices of the GPUs that this actor can use
        # (e.g., [0, 1] or [3, 5]).
        ray.get_gpu_ids()
```

Then inside of the actor constructor and methods, we can get the IDs of the GPUs allocated for that actor with `ray.get_gpu_ids()`.

In [5]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import ray
import time

Start Ray, note that we pass in `num_gpus=4`. Ray will assume this machine has 4 GPUs (even if it does not). When a task or actor requests a GPU, it will be assigned a GPU ID from the set `[0, 1, 2, 3]`. It is then the responsibility of the task or actor to make sure that it only uses that specific GPU (e.g., by setting the `CUDA_VISIBLE_DEVICES` environment variable).

In [21]:
ray.init(num_cpus=4, num_gpus=2, include_webui=False, ignore_reinit_error=True)

2020-05-03 13:09:12,650	INFO resource_spec.py:212 -- Starting Ray with 55.66 GiB memory available for workers and up to 27.83 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-05-03 13:09:12,946	WARNING services.py:1470 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 8589934592 bytes available. This may slow down performance! You may be able to free up space by deleting files in /dev/shm or terminating any running plasma_store_server processes. If you are inside a Docker container, you may need to pass an argument with the flag '--shm-size' to 'docker run'.


{'node_ip_address': '10.68.34.74',
 'redis_address': '10.68.34.74:58561',
 'object_store_address': '/tmp/ray/session_2020-05-03_13-09-12_647438_1459/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-05-03_13-09-12_647438_1459/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2020-05-03_13-09-12_647438_1459'}

**EXERCISE:** Change the remote function below to require one GPU.

**NOTE:** This change does not make the remote function actually **use** the GPU, it simply **reserves** the GPU for use by the remote function. To actually use the GPU, the remote function would use a neural net library like TensorFlow or PyTorch after setting the `CUDA_VISIBLE_DEVICES` environment variable properly. This can be done as follows.

```python
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ','.join([str(i) for i in ray.get_gpu_ids()])
```

In [22]:
@ray.remote(num_gpus=1)
def f():
    time.sleep(0.5)
    return ray.get_gpu_ids()

**VERIFY:** This code checks that each task was assigned one GPU and that not more than two tasks are run at the same time (because we told Ray there are only two GPUs).

In [24]:
start_time = time.time()

gpu_ids = ray.get([f.remote() for _ in range(3)])

end_time = time.time()

print(gpu_ids)

for i in range(len(gpu_ids)):
    assert len(gpu_ids[i]) == 1

assert end_time - start_time > 1

print('Sucess! The test passed.')

[[0], [1], [0]]
Sucess! The test passed.


In [18]:
print((end_time, start_time, end_time-start_time))

(1588510814.6945872, 1588510814.17879, 0.5157971382141113)


**EXERCISE:** The code below defines an actor. Make it require one GPU.

In [25]:
@ray.remote(num_gpus=1)
class Actor(object):
    def __init__(self):
        pass

    def get_gpu_ids(self):
        return ray.get_gpu_ids()

**VERIFY:** This code checks that the actor was assigned a GPU.

In [26]:
actor = Actor.remote()

gpu_ids = ray.get(actor.get_gpu_ids.remote())

assert len(gpu_ids) == 1

print('Sucess! The test passed.')

Sucess! The test passed.
